In [1]:
import numpy as np
import requests
import os
from numpy.linalg import eigh
import matplotlib.pyplot as plt
import mdtraj as md
import nglview as nv
from mpl_toolkits.axes_grid1 import make_axes_locatable


# Working with the Anisotropic Network Model

## Downloading any PDB file

Let's first of all store the PDB id we want to download and the path where the file needs to be located:

In [2]:
pdb_id = '1HIV'
download_path = "./"

And now let's define an auxiliary function to download pdb files to be stored locally in our machine.

In [3]:
def fetch_pdb(pdb_id, download_path="./"):
        url = 'http://files.rcsb.org/download/{}.pdb'.format(pdb_id)
        try:
            res = requests.get(url, allow_redirects=True)
        except:
            print("Could not fetch pdb from {}".format(url))
            return 
        
        file_path = os.path.join(download_path, pdb_id + ".pdb")
        with open(file_path, "wb") as f:
            f.write(res.content)
        print(f"The {pdb_id} file was downloaded succesfully!")

Time to try it!

In [4]:
fetch_pdb(pdb_id, download_path=download_path)

The 1HIV file was downloaded succesfully!


## Parsing the positions of the CA atoms

We are going take an approach not very short and efficient to obtain the coordinates of the CA atoms. But let's do it this way just once:

In [5]:
file_path = os.path.join(download_path, pdb_id + ".pdb")

CA_coordinates = []

with open(file_path) as fff:
    for line in fff.readlines():
        if line.startswith('ATOM  ') and line.split()[2]=='CA' :
          
            x, y, z = line.split()[6:9]
            CA_coordinates.append([float(x), float(y), float(z)])

Ok, `CA_coordinates` is a list of lists... let's convert it to a numpy array:

In [6]:
CA_coordinates = np.array(CA_coordinates)

In [7]:
#Alpha carbon number
len(CA_coordinates)

196

## Defining the equivalent Hessian matrix of the elastic 3D model

In [12]:
def anm_matrix(CA, threshold, force_constant):
    
    num_CAs = CA.shape[0]
    threshold2 = threshold**2
    
    matrix = np.zeros((3*num_CAs, 3*num_CAs), dtype=float)
    
    
    
    for ii in range(num_CAs):
        
        iii = ii*3
        
        for jj in range(ii):
            
            jjj = jj*3
            
            rij=CA[ii]-CA[jj]
            distance2=np.dot(rij, rij)
                
            if distance2<threshold2:
                    
                for kk in range(3):
                    for gg in range(3):
                        
                        val_aux = -(CA[jj][kk]-CA[ii][kk])*(CA[jj][gg]-CA[ii][gg])/distance2
                        
                        ## Sub matrix Hij
                        matrix[iii+kk,jjj+gg]=val_aux
                        matrix[jjj+gg,iii+kk]=val_aux
                        
                        #Submatrix Hii:
                        matrix[iii+kk,iii+gg]-=val_aux
                        matrix[jjj+gg,jjj+kk]-=val_aux
                        
    return force_constant*matrix

In [13]:
matrix = anm_matrix(CA_coordinates, 9.0, 1.0)
matrix

array([[ 2.43097802e+00, -3.38042889e-01, -1.14520061e+00, ...,
        -4.02070664e-04,  1.23636729e-02, -1.57812736e-02],
       [-3.38042889e-01,  9.30784792e-01, -5.44984519e-01, ...,
         1.23636729e-02, -3.80182942e-01,  4.85274162e-01],
       [-1.14520061e+00, -5.44984519e-01,  1.63823719e+00, ...,
        -1.57812736e-02,  4.85274162e-01, -6.19414987e-01],
       ...,
       [-4.02070664e-04,  1.23636729e-02, -1.57812736e-02, ...,
         3.33775970e+00, -1.41481185e+00,  3.96181163e-01],
       [ 1.23636729e-02, -3.80182942e-01,  4.85274162e-01, ...,
        -1.41481185e+00,  2.15223524e+00, -1.70378695e+00],
       [-1.57812736e-02,  4.85274162e-01, -6.19414987e-01, ...,
         3.96181163e-01, -1.70378695e+00,  4.51000506e+00]])

## Diagonalizing from its eigenvalues

In [15]:
evals, evects = eigh(matrix)

In [16]:
evals

array([-1.10906672e-15, -9.30575126e-16, -6.46171550e-16, -1.99110724e-16,
        5.74767471e-16,  8.31176694e-16,  1.41310837e-02,  1.95065219e-02,
        2.56914126e-02,  2.96869723e-02,  4.34234123e-02,  5.73280130e-02,
        6.77178531e-02,  7.27184883e-02,  7.66837138e-02,  8.57226988e-02,
        9.81058000e-02,  1.02745903e-01,  1.05696426e-01,  1.14463925e-01,
        1.19124011e-01,  1.33582979e-01,  1.38074033e-01,  1.42891005e-01,
        1.68101054e-01,  1.73619126e-01,  1.79950238e-01,  2.00691043e-01,
        2.14924831e-01,  2.23472149e-01,  2.39132459e-01,  2.44397976e-01,
        2.51889208e-01,  2.63083752e-01,  2.88781574e-01,  2.93131061e-01,
        3.01946383e-01,  3.10923666e-01,  3.23406114e-01,  3.38165150e-01,
        3.46670599e-01,  3.57991454e-01,  3.64028531e-01,  3.93782325e-01,
        4.15194965e-01,  4.33639934e-01,  4.47287427e-01,  4.59725680e-01,
        4.60162555e-01,  4.87650777e-01,  5.03019625e-01,  5.18526759e-01,
        5.23976188e-01,  

In [ ]:
num_modes = evals.shape[0]

plt.scatter(range(num_modes), evals)
plt.title('Eigenvalues Spectrum')
plt.xlabel('Eigenvalue index')
plt.show()

Ahora los modos normales contienen una componente $x$, $y$ y $z$ para cada CA. ¿Cómo podemos hacer una representación similar a la que hacíamos en el GNM para ver la amplitud participación de cada CA en el modo? ¿Qué tal si representamos algo como $\sqrt{x_{i}^{2} + y_{i}^{2} + z_{i}^{2}}$ para cada CA de índice $i$ en un mismo modo?

In [ ]:
# Esto viene de GNM y debe ser adaptado para ANM
mode_index = 0
plt.plot(range(num_modes), evects[:,mode_index])
plt.title(f'{mode_index}-th normalized mode')
plt.xlabel('CA index')
plt.show()